In [1]:
import sys
import os
sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca\\pointing')
sys.path.append('D:\\software_git_repos\\greenpol')
sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\')
sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\VtoT\\')
import realtime_gp as rt
import numpy as np
import datetime 
import h5py
import pandas as pd
from pointingtools import compute_parallactic_angle, altaz2ha 
from planets import getlocation
import warnings
from astropy.coordinates import AltAz, Angle, EarthLocation, ICRS, SkyCoord, frame_transform_graph
from astropy import units as u
import ephem
import matplotlib.pyplot as plt
import time
import calendar
import realtime_gp as rt

import Tkinter,tkFileDialog

from Tkinter import *
import ttk

from scipy import optimize
import pickle

import numpy.ma as ma

import cPickle


%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Widget', 'Text', 'Button']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
def nametochan(name):
    #function to convert channel numbers to channel names

    #names of each channel
    chans = {
        'all': 'all',  'H1HiAC':'ch0',  'H1HiDC':'ch1',
        'H1LoAC':'ch2' ,  'H1LoDC':'ch3', 'H2HiAC':'ch4' ,
        'H2HiDC':'ch5' ,  'H2LoAC':'ch6',  'H2LoDC':'ch7',
        'H3HiAC':'ch8', 'H3HiDC':'ch9',  'H3LoAC':'ch10',
         'H3LoDC':'ch11', 'HornTop':'ch12', 'Amplifier': 'ch13',
        'Cooler':'ch14', 'Transition':'ch15'}

    chan = chans[name]

    return chan

In [2]:
a = 0
if a == True:
    print 'hey'
else:
    print 'ney'

ney


In [3]:
def get_pointing_files(datadict,datalabel, filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/pointing_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
    return filelist

In [4]:
def read_some_data(datadict,datalabel, filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
    dlist=[]
    for f in filelist:
        hf=h5py.File(f)
        dlist.append(hf['demod_data'])
    d=np.concatenate(dlist)
    hf.close() 

    datadict[datalabel]=d
    return

In [6]:
def combine_cofe_h5_pointing(dd,h5pointing,outfile='combined_data.pkl'):
    """
    combine demodulated data (output of get_demodulated_data_from_list) with H5 pointing data 
    (output of get_h5_pointing), dump to a pkl file by default (this takes a long time to run)
    """
    paz=h5pointing['az'].copy()
    prev=h5pointing['gpstime'].copy()

    #need to use only 24 bits for comparison with science data gpstime- probably a better way to do this, bitmasking?
    prev  &= 0x00ffffff
    pazw=np.where(abs(np.diff(paz) + 359.5)  < 180.)[0]   #find the wrapping points so we can unroll the az for interpolation
    for r in pazw:
        paz[r+1:]=paz[r+1:]+360.                   #unwrap az
    azout=np.interp(dd['rev'],prev,paz)
    azout=np.mod(azout,360.)
    elout=np.interp(dd['rev'],prev,h5pointing['el'])
    gpstimeout = np.interp(dd['rev'],prev,prev) #+ 419430400

    f=open(outfile,'wb')
    combined_data={'sci_data':dd,'az': azout,'el':elout, 'gpstime': gpstimeout}
    cPickle.dump(combined_data,f,protocol=-1)
    f.close()
    return combined_data

In [10]:
el_bits=[43,18,42,17,41,16,40,15,39,91,66,90,65,89,64,88,63,87]
az_bits=[23,47,22,46,21,45,20,44,71,95,70,94,69,93,68,92]
gps_bits=[24,48,0,73,25,49,1,74,26,50,2,75,27,51,3,76,28,52,4,77,29,53,5,78,30,54,6,79,31,55,7,80]
bits = el_bits+az_bits+gps_bits
print 96-len(bits)
xbits = []
for b in range(0,96):
    if b not in bits:
        xbits.append(b)
print xbits

30
[8, 9, 10, 11, 12, 13, 14, 19, 32, 33, 34, 35, 36, 37, 38, 56, 57, 58, 59, 60, 61, 62, 67, 72, 81, 82, 83, 84, 85, 86]


In [8]:
ddict={}
pdict={}

In [ ]:
read_some_data(ddict,'test')

In [29]:
flp = get_pointing_files(pdict, 'test')

In [30]:
#dd = ddict['test']
pp = rt.get_h5_pointing(flp)

In [31]:
plot(pp['gpstime'])

In [20]:
t = pp['gpstime']/1000
sps = float(len(t))/(t[-1]-t[0])
print 'samples per second: ', sps

samples per second:  0.997635933806


In [21]:
combined = combine_cofe_h5_pointing(dd,pp)

In [22]:
plt.plot(pp['az'])

In [23]:
chan = 'ch1'
var = 'T'
xaxis = 'az'

In [24]:
xa = combined[xaxis]
data = combined['sci_data'][chan][var]

#sort data according to sorted azimuth
data = [x for _,x in sorted(zip(xa,data))]
xa = sorted(xa)

In [25]:
plot(xa, data)

In [85]:
minval = np.zeros(len(pp['az']))
minval[:] = pp['az'].min()
plt.plot(pp['az'], 'b-')
plt.plot(abs(np.diff(pp['az'])), 'g-')
plt.plot(minval, 'r-')
plt.show()

In [159]:
i = 0
plot(az[i], data[i])
i = 6
plot(az[i], data[i])

In [26]:
#synchronized data and az values
az1 = combined['az']
data1 = combined['sci_data'][chan][var]
steps = len(data1)

#data1 = np.asarray([x for _,x in sorted(zip(az1,data1))])
#az1 = sorted(az1)

#convert to temp for cryo sensors
if chan == 12:
    data1 = convert.convert(data1, 'i')
if chan == 13:
    data1 = convert.convert(data1, 'e')
if chan == 14:
    data1 = convert.convert(data1, 'h')
if chan == 15:
    data1 = convert.convert(data1, 'l')

#resolution
dx = 1.0
dy = 1.0

#set up empty lists to append each revolution to
data = []
az = []
iaz = [0]
iaz2 = [0]
rev = 0

#determine indices in azimuth/data array which correspond to a new revolution of the telescope
for i in range(steps):
    #round values to resolution for comparison later
    az1[i] = rt.round_fraction(az1[i], dx)
    if i > 0:
        if abs(az1[i] - az1[i-1]) >= 180.0:
            iaz.append(i)
            rev += 1

#append each revolution array to a list	    
for j in range(rev):
    az.append(az1[iaz[j]:iaz[j+1]])
    data.append(data1[iaz[j]:iaz[j+1]])

#append the last revolution
data.append(data1[iaz[-1]:])
az.append(az1[iaz[-1]:])
rev += 1

data = np.asarray(data)
az = np.asarray(az)
#create grid for plotting
x, y = np.arange(0., 360.+dx, dx), np.arange(0., rev - 1 + dy, dy)
AZ, REV = np.meshgrid(x, y)

#set up empty array
z = np.zeros(len(x)*len(y))
sig = np.reshape(z, (len(y), len(x)))

#small number for comparing floats
epsilon = 1e-6

#fill signal array with data points
for r in range(rev):
    for a in range(len(x)):
        #find indices where combined azimuth data fits on x grid
        idx = np.where(abs(az[r] - x[a]) < epsilon)[0]
        #if idx length is 0 this will create a mask on that point, in idx len > 1, avg data points in the same bin
        sig[r][a] = data[r][idx].mean()

#mask invalid values, i.e. where there are no data points
sig = ma.masked_invalid(sig)

#change units on plot label
if int(chan[2:]) < 12:
    unit = 'V'
else:
    unit = 'K' 

name = rt.chantoname(chan)
plt.pcolormesh(AZ, REV, sig)
plt.colorbar(label = 'Signal, %s' % unit)
plt.clim(data1.min(),data1.max())
plt.axis([0., 360., 0., rev - 1])
plt.ylabel('revolution #')
plt.xlabel('azimuth (deg)')
#plt.title('%s %s data binned to azimuth and revolution #, date %s, %s:%d - %s:%d' % (name, var, yrmoday, st_hour, int(st_minute), ed_hour, int(ed_minute)))
plt.grid()
plt.show()

C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\numpy\core\_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
